# #6. Django: Origins

0. [Motivation](#Motivation)
1. [Dynamic Web](#Dynamic-Web)
    1. [CGI](#CGI)
    2. [FastCGI](#FastCGI)
    3. [WSGI](#WSGI)
    4. [Gunicorn](#Gunicorn)
    5. [uWSGI](#uWSGI)
2. [Tiered architecture](#Tiered-architecture)
3. [Django](#Django)
    1. [MVC](#MVC)
    2. [Coding style](#Coding-style)
    3. [Enviroment setup](#Enviroment-setup)
    3. [Project structure](#Project-structure)
    4. [Configuration](#Configuration)
    5. [Dependencies](#Dependencies)
    6. [URL dispatching](#URL-dispatching)

## Motivation

__Goal__

Разобраться с тем, как запускаются и разварачиваются веб-приложения и плавно перейти к Django

__Homework__

Django tutorial or uWSGI daemon

## Dynamic Web

#### CGI

The Common Gateway Interface (CGI) is a standardized interface for delegating web requests to external applications that handle the request and generate a response
* start new process
    * very costly 
* write request to STDIN
* set ENV
* read response from STDOUT
    * should be on the same machine

![cgi](https://www.electricmonk.nl/docs/apache_fastcgi_python/flow_cgi.png)

In [ ]:
#!/usr/bin/env python
import os

query = os.environ['QUERY_STRING']
if os.environ['REQUEST_METHOD'] == 'POST':
    size = int(os.environ['CONTENT_LENGTH'])
    query = sys.stdin.read(size)

In [1]:
#!/usr/bin/env python

import cgi
import cgitb; cgitb.enable()  # for troubleshooting

print "Content-type: text/html"
print

print """
<html>

<head><title>Sample CGI Script</title></head>

<body>

  <h3> Sample CGI Script </h3>
"""

form = cgi.FieldStorage()
message = form.getvalue("message", "(no message)")

print """

  <p>Previous message: %s</p>

  <p>form

  <form method="post" action="index.cgi">
    <p>message: <input type="text" name="message"/></p>
  </form>

</body>

</html>
""" % cgi.escape(message)

Content-type: text/html


<html>

<head><title>Sample CGI Script</title></head>

<body>

  <h3> Sample CGI Script </h3>



  <p>Previous message: (no message)</p>

  <p>form

  <form method="post" action="index.cgi">
    <p>message: <input type="text" name="message"/></p>
  </form>

</body>

</html>



#### FastCGI

FastCGI mitigates the main issues of CGI by specifying an interface protocol to be used via local sockets or TCP connections. Instead of creating a new process for each request, FastCGI uses persistent processes to handle a series of requests. These processes are owned by the FastCGI server, not the web server.

![fast](https://www.electricmonk.nl/docs/apache_fastcgi_python/flow_fastcgi.png)

In [ ]:
location ~ \.py$ {
    include /etc/nginx/fcgi_params;
    fastcgi_pass  127.0.0.1:9000;
}

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-

from cgi import escape
import sys, os
from flup.server.fcgi import WSGIServer

def app(environ, start_response):
    start_response('200 OK', [('Content-Type', 'text/html')])

    yield '<h1>FastCGI Environment</h1>'
    yield '<table>'
    for k, v in sorted(environ.items()):
        yield '<tr><th>%s</th><td>%s</td></tr>' % (escape(k), escape(v))
    yield '</table>'

WSGIServer(app).run()

#### WSGI

![wsgi](https://ruslanspivak.com/lsbaws-part2/lsbaws_part2_wsgi_interface.png)

In [6]:
# The application interface is a callable object
def application ( # It accepts two arguments:
    # environ points to a dictionary containing CGI like environment
    # variables which is populated by the server for each
    # received request from the client
    environ,
    # start_response is a callback function supplied by the server
    # which takes the HTTP status and headers as arguments
    start_response
):

    # Build the response body possibly
    # using the supplied environ dictionary
    response_body = 'Request method: %s' % environ['REQUEST_METHOD']

    # HTTP response code and message
    status = '200 OK'

    # HTTP headers expected by the client
    # They must be wrapped as a list of tupled pairs:
    # [(Header name, Header value)].
    response_headers = [
        ('Content-Type', 'text/plain'),
        ('Content-Length', str(len(response_body)))
    ]

    # Send them to the server using the supplied function
    start_response(status, response_headers)

    # Return the response body. Notice it is wrapped
    # in a list although it could be any iterable.
    return [response_body]

In [13]:
# Tested with Python 2.7.9, Linux & Mac OS X
import socket
import StringIO
import sys


class WSGIServer(object):

    address_family = socket.AF_INET
    socket_type = socket.SOCK_STREAM
    request_queue_size = 1

    def __init__(self, server_address):
        # Create a listening socket
        self.listen_socket = listen_socket = socket.socket(
            self.address_family,
            self.socket_type
        )
        # Allow to reuse the same address
        listen_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        # Bind
        listen_socket.bind(server_address)
        # Activate
        listen_socket.listen(self.request_queue_size)
        # Get server host name and port
        host, port = self.listen_socket.getsockname()[:2]
        self.server_name = socket.getfqdn(host)
        self.server_port = port
        # Return headers set by Web framework/Web application
        self.headers_set = []

    def set_app(self, application):
        self.application = application

    def serve_forever(self):
        listen_socket = self.listen_socket
        while True:
            # New client connection
            self.client_connection, client_address = listen_socket.accept()
            # Handle one request and close the client connection. Then
            # loop over to wait for another client connection
            self.handle_one_request()

    def handle_one_request(self):
        self.request_data = request_data = self.client_connection.recv(1024)
        # Print formatted request data a la 'curl -v'
        print(''.join(
            '< {line}\n'.format(line=line)
            for line in request_data.splitlines()
        ))

        self.parse_request(request_data)

        # Construct environment dictionary using request data
        env = self.get_environ()

        # It's time to call our application callable and get
        # back a result that will become HTTP response body
        result = self.application(env, self.start_response)

        # Construct a response and send it back to the client
        self.finish_response(result)

    def parse_request(self, text):
        request_line = text.splitlines()[0]
        request_line = request_line.rstrip('\r\n')
        # Break down the request line into components
        (self.request_method,  # GET
         self.path,            # /hello
         self.request_version  # HTTP/1.1
         ) = request_line.split()

    def get_environ(self):
        env = {}
        # The following code snippet does not follow PEP8 conventions
        # but it's formatted the way it is for demonstration purposes
        # to emphasize the required variables and their values
        #
        # Required WSGI variables
        env['wsgi.version']      = (1, 0)
        env['wsgi.url_scheme']   = 'http'
        env['wsgi.input']        = StringIO.StringIO(self.request_data)
        env['wsgi.errors']       = sys.stderr
        env['wsgi.multithread']  = False
        env['wsgi.multiprocess'] = False
        env['wsgi.run_once']     = False
        # Required CGI variables
        env['REQUEST_METHOD']    = self.request_method    # GET
        env['PATH_INFO']         = self.path              # /hello
        env['SERVER_NAME']       = self.server_name       # localhost
        env['SERVER_PORT']       = str(self.server_port)  # 8888
        return env

    def start_response(self, status, response_headers, exc_info=None):
        # Add necessary server headers
        server_headers = [
            ('Date', 'Tue, 31 Mar 2015 12:54:48 GMT'),
            ('Server', 'WSGIServer 0.2'),
        ]
        self.headers_set = [status, response_headers + server_headers]
        # To adhere to WSGI specification the start_response must return
        # a 'write' callable. For simplicity's sake we'll ignore that detail
        # for now.
        # return self.finish_response

    def finish_response(self, result):
        try:
            status, response_headers = self.headers_set
            response = 'HTTP/1.1 {status}\r\n'.format(status=status)
            for header in response_headers:
                response += '{0}: {1}\r\n'.format(*header)
            response += '\r\n'
            for data in result:
                response += data
            # Print formatted response data a la 'curl -v'
            print(''.join(
                '> {line}\n'.format(line=line)
                for line in response.splitlines()
            ))
            self.client_connection.sendall(response)
        finally:
            self.client_connection.close()


SERVER_ADDRESS = (HOST, PORT) = '', 8888


def make_server(server_address, application):
    server = WSGIServer(server_address)
    server.set_app(application)
    return server


if __name__ == '__main__':
    if len(sys.argv) < 2:
        sys.exit('Provide a WSGI application object as module:callable')
    app_path = sys.argv[1]
    module, application = app_path.split(':')
    module = __import__(module)
    application = getattr(module, application)
    httpd = make_server(SERVER_ADDRESS, application)
    print('WSGIServer: Serving HTTP on port {port} ...\n'.format(port=PORT))
    httpd.serve_forever()

* WSGI stands for (Web Server Gateway Inteface)
* WSGI container is a separate running process that runs on a different port than your web server
* web server is configured to pass requests to the WSGI container which runs web application, then pass the response (in the form of HTML) back to the requester

![wsgi2](https://ruslanspivak.com/lsbaws-part2/lsbaws_part2_server_summary.png)

In [9]:
#! /usr/bin/env python

from wsgiref.simple_server import make_server

def application(environ, start_response):

    response_body = [
        '%s: %s' % (key, value) for key, value in sorted(environ.items())
    ]
    response_body = '\n'.join(response_body)

    # Adding strings to the response body
    response_body = [
        'The Beggining\n',
        '*' * 30 + '\n',
        response_body,
        '\n' + '*' * 30 ,
        '\nThe End'
    ]

    # So the content-lenght is the sum of all string's lengths
    content_length = sum([len(s) for s in response_body])

    status = '200 OK'
    response_headers = [
        ('Content-Type', 'text/plain'),
        ('Content-Length', str(content_length))
    ]

    start_response(status, response_headers)
    return [response_body]

httpd = make_server('localhost', 8042, application)
httpd.handle_request()

In [11]:
! curl http://localhost:8042/

The Beggining
******************************
Apple_PubSub_Socket_Render: /private/tmp/com.apple.launchd.n0Zz34ihUb/Render
CLICOLOR: 1
COLORFGBG: 7;0
COMMAND_MODE: unix2003
CONTENT_LENGTH: 
CONTENT_TYPE: text/plain
GATEWAY_INTERFACE: CGI/1.1
HOME: /Users/s.stupnikov
HTTP_ACCEPT: */*
HTTP_HOST: localhost:8042
HTTP_USER_AGENT: curl/7.54.0
IRC_CLIENT: irssi
ITERM_PROFILE: s.stupnikov
ITERM_SESSION_ID: w0t1p0:636D6600-1F7B-4ABA-AF12-59DDDF2CE23F
LANG: ru_RU.UTF-8
LOGNAME: s.stupnikov
LSCOLORS: GxFxCxDxBxegedabagaced
PATH: /opt/local/bin:/opt/local/sbin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/bin
PATH_INFO: /
PS1: \[\033\]\u@\h\[\033\]:\[\033\]\w\[\033\]\$ 
PWD: /Users/s.stupnikov/Coding
QUERY_STRING: 
REMOTE_ADDR: 127.0.0.1
REMOTE_HOST: 1.0.0.127.in-addr.arpa
REQUEST_METHOD: GET
SCRIPT_NAME: 
SECURITYSESSIONID: 186a7
SERVER_NAME: 1.0.0.127.in-addr.arpa
SERVER_PORT: 8042
SERVER_PROTOCOL: HTTP/1.1
SERVER_SOFTWARE: WSGIServer/0.1 Python/2.7.10
SHELL: /bin/bash
SHLVL: 1
SSH_AUTH_SOCK: /private

In [12]:
from cgi import parse_qs, escape

def application(environ, start_response):

    # the environment variable CONTENT_LENGTH may be empty or missing
    try:
        request_body_size = int(environ.get('CONTENT_LENGTH', 0))
    except (ValueError):
        request_body_size = 0

    # When the method is POST the variable will be sent
    # in the HTTP request body which is passed by the WSGI server
    # in the file like wsgi.input environment variable.
    request_body = environ['wsgi.input'].read(request_body_size)
    d = parse_qs(request_body)

    age = d.get('age', [''])[0] # Returns the first age value.
    hobbies = d.get('hobbies', []) # Returns a list of hobbies.

    # Always escape user input to avoid script injection
    age = escape(age)
    hobbies = [escape(hobby) for hobby in hobbies]

    response_body = html % { # Fill the above html template in
        'checked-software': ('', 'checked')['software' in hobbies],
        'checked-tunning': ('', 'checked')['tunning' in hobbies],
        'age': age or 'Empty',
        'hobbies': ', '.join(hobbies or ['No Hobbies?'])
    }

    status = '200 OK'

    response_headers = [
        ('Content-Type', 'text/html'),
        ('Content-Length', str(len(response_body)))
    ]

    start_response(status, response_headers)
    return [response_body]

Middleware is an application that calls "the next" application; functional composition forms a "pipeline".

In [ ]:
class LowercaseMiddleware:
    def __init__(self, application):
        self.application = application   # A WSGI application callable.

    def __call__(self, environ, start_response):
        pass  # We could set an item in 'environ' or a local variable.
        for chunk in self.application(environ, start_response):
            yield chunk.lower()

app = LowercaseMiddleware(application)

In [ ]:
from raven import Client
from raven.middleware import Sentry

application = Sentry(
    application,
    Client('https://<key>:<secret>@sentry.io/<project>')
)

#### Gunicorn

Gunicorn is based on the pre-fork worker model. This means that there is a central master process that manages a set of worker processes. The master never knows anything about individual clients. All requests and responses are handled completely by worker processes.

Workers:
* sync
* async
* tornado

In [ ]:
gunicorn -b 0.0.0.0:8080 --workers=5 --name my_application --log-file error_logs.log --access-logfile acclogs wsgi

In [ ]:
worker_processes 1;

events {

    worker_connections 1024;

}

http {

    sendfile on;

    gzip              on;

    # Configuration containing list of application servers
    upstream app_servers {

        server 127.0.0.1:8080;
        # server 127.0.0.1:8081;
        # ..
        # .

    }

    # Configuration for Nginx
    server {

        # Running port
        listen 80;

        # Settings to serve static files 
        location ^~ /static/  {
            root /app/static/;

        }

        # Proxy connections to the application servers
        # app_servers
        location / {

            proxy_pass         http://app_servers;
            proxy_redirect     off;
            proxy_set_header   Host $host;
            proxy_set_header   X-Real-IP $remote_addr;
            proxy_set_header   X-Forwarded-For $proxy_add_x_forwarded_for;
            proxy_set_header   X-Forwarded-Host $server_name;

        }
    }
}

#### uWSGI

![uwsgi1](http://zarnovican.github.io/static/2016/02/15/pre-fork.png)

Emperor

![uwsgi2](http://zarnovican.github.io/static/2016/02/15/emperor-mode.png)

Chain-restart
* master will gracefully shutdown one worker at a time (ordered by id apparently)
* when a worker dies, it is immediately re-spawned
* until all workers are re-spawned
* not compatible with pre-fork
* no downtime!

![uwsgi3](http://zarnovican.github.io/static/2016/02/15/reload-c.png)

Fastrouter

In [ ]:
        location / {
            set $route "common";

            if ($uri ~ ^/api/) {
                set $route "api";
            }

            if ($uri ~ ^/(forecast|service/stat)/) {
                set $route "slow";
            }

            if ($cookie_rb_stage) {
                set $route $cookie_rb_stage;
            }

            include uwsgi_params;
            uwsgi_param UWSGI_FASTROUTER_KEY $route;
            uwsgi_read_timeout 1800;
            uwsgi_pass backend_servers;  # rbui1
        }

### References

* https://tools.ietf.org/html/draft-robinson-www-interface-00
* http://www.nongnu.org/fastcgi/
* http://www.mit.edu/~yandros/doc/specs/fcgi-spec.html
* https://docs.python.org/2/howto/webservers.html
* http://www.dabeaz.com/python/PythonNetBinder.pdf
* http://berb.github.io/diploma-thesis/original/031_tradwebarch.html
* https://www.python.org/dev/peps/pep-0333/
* https://www.fullstackpython.com/wsgi-servers.html
* http://wsgi.tutorial.codepoint.net/intro
* https://ruslanspivak.com/lsbaws-part2/
* http://zarnovican.github.io/2016/02/15/uwsgi-graceful-reload/
* https://www.haproxy.com/blog/truly-seamless-reloads-with-haproxy-no-more-hacks/

### Summary

* for Python use WSGI
* use Nginx
* study uWSGI doc

## Tiered architecture

![arch](http://berb.github.io/diploma-thesis/original/resources/web_arch.svg)

![arch2](https://www.sayonetech.com/media/2016/09/22/instagram.jpg)

![arch4](https://www.sayonetech.com/media/2016/09/20/disqus.jpg)

![arch3](https://www.sayonetech.com/media/2016/09/20/pinterest.jpg)

Start point
![simple](https://i.onthe.io/shpzkl4gjvlbac0l3.1d52a7d7.jpg)

### References

* http://berb.github.io/diploma-thesis/original/033_archmodel.html
* https://www.sayonetech.com/blog/software-stack-five-hot-startups-running-django/#.WX-DP9PyjdQ
* https://www.insight-it.ru/highload/2012/arkhitektura-instagram/
* https://stackshare.io/stacks
* http://highload.guide/
* https://park.mail.ru/materials/video/#31

### Summary

* Tiered acrchitecture is a current best practice

## Django

__Design philosophies__

* Loose coupling
* Less code
* Quick development
* DRY
* Explicit is better than implicit
* Consistency

#### MVC

![mtv](http://www.thomaswhitton.com/django-presentation/images/432038560_9f8b830dfe_o.png)

![cycle](http://rnevius.github.io/django_request_response_cycle.png)

#### Coding style

* PEP8
    * Maximum line length
* Imports order
* Explicit relative imports

In [ ]:
# future
from __future__ import unicode_literals

# standard library
import json
from itertools import chain

# third-party
import bcrypt

# Django
from django.http import Http404
from django.http.response import (
    Http404, HttpResponse, HttpResponseNotAllowed, StreamingHttpResponse,
    cookie,
)

# local Django
from .models import LogEntry

# try/except
try:
    import yaml
except ImportError:
    yaml = None

CONSTANT = 'foo'

In [ ]:
# banners/views.py
from django.views.generic import CreateView

from .models import Banner
from .forms import BannerForm
from core.views import AdMixin

class BannerCreateView(AdMixin, CreateView):
    model = Banner
    form_class = BannerForm

Use underscore in URL pattern names

In [ ]:
patterns = [
    url(regex='^add/$',
        view=views.add_banner,
        name='add_banner'),
    ]

#### Enviroment setup

* keep the gap between development and production small
    * example: dont use SQLite for dev and other DB for prod
* never rely on implicit existence of system-wide packages
    * declare all dependencies
    * use dependency isolation

In [ ]:
pip install virtualenvwrapper
export WORKON_HOME=~/Envs
source /usr/local/bin/virtualenvwrapper.sh


mkvirtualenv my_project
workon my_project

#### Project structure

#### Configuration

In [ ]:
# local.py

from .base import *

DEBUG = True

EMAIL_BACKEND = 'django.core.mail.backends.console.EmailBackend'

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql_psycopg2',
        'NAME': 'uber',
        'HOST': 'localhost',
    }
}

INSTALLED_APPS += ['debug_toolbar', ]

In [ ]:
export DJANGO_SETTINGS_MODULE=mysite.settings
django-admin runserver

DJANGO_SETTINGS_MODULE=mysite.settings django-admin runserver

django-admin runserver --settings=mysite.settings

__Secrets__

In [ ]:
# settings/base.py
import os

# Normally you should not import ANYTHING from Django directly
# into your settings, but ImproperlyConfigured is an exception.
from django.core.exceptions import ImproperlyConfigured

def get_env_variable(var_name):
    """Get the environment variable or return exception."""
    try:
        return os.environ[var_name]
    except KeyError:
        error_msg = 'Set the {} environment variable'.format(var_name)
        raise ImproperlyConfigured(error_msg)


In [ ]:
# Top of settings/production.py
import os
SOME_SECRET_KEY = get_env_variable('SOME_SECRET_KEY')

In [ ]:
# settings/base.py
import json

from django.core.exceptions import ImproperlyConfigured

# JSON-based secrets module
with open('secrets.json') as f:
    secrets = json.loads(f.read())

def get_secret(setting, secrets=secrets):
    '''Get the secret variable or return explicit exception.'''
    try:
        return secrets[setting]
    except KeyError:
        error_msg = 'Set the {0} environment variable'.format(setting)
        raise ImproperlyConfigured(error_msg)

SECRET_KEY = get_secret('SECRET_KEY')

__Paths__

In [ ]:
# At the top of settings/base.py

from os.path import abspath, dirname, join

def root(*dirs):
    base_dir = join(dirname(__file__), '..', '..')
    return abspath(join(bas_dir, *dirs))


BASE_DIR = root()
MEDIA_ROOT = root('media')
STATIC_ROOT = root('static_root')
STATICFILES_DIRS = [root('static')]
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [root('templates')],
    },
]

#### Dependencies

In [ ]:
pip install -r requirements/local.txt

#### URL dispatching

* Django determines the root URLconf module to use. Ordinarily, this is the value of the ROOT_URLCONF setting
* Django loads that Python module and looks for the variable urlpatterns.
* Django runs through each URL pattern, in order, and stops at the first one that matches the requested URL.
* Once one of the regexes matches, Django imports and calls the given view, which is a simple Python function (or a class-based view). The view gets passed the following arguments:
    * An instance of HttpRequest.
    * If the matched regular expression returned no named groups, then the matches from the regular expression are provided as positional arguments.
    * The keyword arguments are made up of any named groups matched by the regular expression, overridden by any arguments specified in the optional kwargs argument to django.conf.urls.url().
* If no regex matches, or if an exception is raised during any point in this process, Django invokes an appropriate error-handling view. 

In [ ]:
# project urls.py

from django.conf.urls import include, url

urlpatterns = [
    url(r'^author-polls/', include('polls.urls', namespace='author-polls')),
    url(r'^publisher-polls/', include('polls.urls', namespace='publisher-polls')),
]

In [ ]:
# polls/urls.py

from django.conf.urls import url

from . import views

app_name = 'polls'
urlpatterns = [
    url(r'^$', views.IndexView.as_view(), name='index'),
    url(r'^(?P<pk>\d+)/$', views.DetailView.as_view(), name='detail'),
]

### References

* https://docs.djangoproject.com/en/1.11/misc/design-philosophies/
* https://docs.djangoproject.com/en/1.11/internals/contributing/writing-code/coding-style/
* https://docs.djangoproject.com/en/1.11/internals/
* https://www.python.org/dev/peps/pep-0328/
* https://12factor.net/
* http://python-guide-pt-br.readthedocs.io/en/latest/dev/virtualenvs/
* http://cookiecutter-django.readthedocs.io/en/latest/developing-locally-docker.html
* http://www.eidel.io/2017/07/10/dockerizing-django-uwsgi-postgres/
* https://www.kennethreitz.org/essays/a-better-pip-workflow
* https://github.com/pydanny/cookiecutter-django
* https://docs.djangoproject.com/en/1.11/topics/http/urls/
* http://klen.github.io/py-frameworks-bench/

### Summary

* Django follows MVC pattern
* Isolate your enviroment
* Split config
* Use enviroment variables
* Freeze dependencies